In [1]:
import numpy as np
import pandas as pd
import math as m
import random as r
import queue
import ipywidgets as widgets
# > <

In [2]:
def get_element_position(state,value):
    rows = len(state)
    columns = len(state[0])
    for row in range(0,rows):
        for column in range(0, columns):
            if(value == state[row][column]):
                return row,column

def swap_elements_by_position(state,el1,el2):
    tempval = state[el1[0]][el1[1]]
    state[el1[0]][el1[1]] = state[el2[0]][el2[1]]
    state[el2[0]][el2[1]] = tempval
    return state
def shiftList(list_):
    return np.flip(list_,axis=0).tolist()

In [3]:
checked_positions = []
obstacles_positions = []
class Board:
    global checked_positions
    global obstacles_positions
    def update_board(self):
        #print(str(self.initial_state)+" "+str(self.goal_state)+"BOARD STATE: "+str(self.board_state))
        x_ini = int(self.initial_state[0])
        y_ini = int(self.initial_state[1])
        x_gol = int(self.goal_state[0])
        y_gol = int(self.goal_state[1])
        
        for checked in checked_positions:
            self.board_state[checked[0]][checked[1]] = 1
        for obstacle in obstacles_positions:
            self.board_state[obstacle[0]][obstacle[1]] = 2
            
        #print(str(x_ini)+" "+str(y_ini)+"- Init - GOAL - "+str(x_gol)+" "+str(y_gol))
        self.board_state[x_ini][y_ini] = 8
        self.board_state[x_gol][y_gol] = 9

        checked_positions.append([x_ini,y_ini])
    
    def __init__(self, width, height, initial_state, goal_state):
        self.width  = width
        self.height = height
        self.board_state   = np.array(list([[0]*width]*height))

        self.initial_state = initial_state
        self.goal_state    = goal_state

        self.update_board()
        print(checked_positions)
        
    def set_random_obstacles(self,block_rate):
        #print("blockrate = "+str(block_rate))
        if(block_rate>0):
            block_rate/=100
            
        size = self.width*self.height
        n_obstacles = m.floor(block_rate*size)
        
        print("n_obstacles = "+str(n_obstacles))
        obstacles_count = 0
        
        while obstacles_count < n_obstacles:
            x = r.randint(0,int(self.height-1))
            y = r.randint(0,int(self.width-1))
            
            #print("board["+str(x)+"]["+str(y)+"]")
            act_val = self.board_state[x][y]
            
            if act_val == 0:
                self.board_state[x][y] = 2
                obstacles_positions.append([x,y])
                obstacles_count+=1
    
    #Print the board
    def __str__(self):
        self.update_board()
        display(pd.DataFrame(self.board_state))
        return str(pd.DataFrame(self.board_state))
    

In [4]:
def addtoQueue(queue_,list_of_boards):
    if len(list_of_boards) == 0 or len(list_of_boards[0]) == 0 :        
        return
    
    for b in list_of_boards:
        b_notExist = get_element_position(b,9)
        if b_notExist is None:
            temp_b = Board(width=len(b[0]), height=len(b), 
                       initial_state=list(get_element_position(b,8)), 
                       goal_state=list(get_element_position(b,8)))
        else:
            temp_b = Board(width=len(b[0]), height=len(b), 
                           initial_state=list(get_element_position(b,8)), 
                           goal_state=list(get_element_position(b,9)))
        queue_.put(temp_b)
        
    
def addtoQueueL(queue_,list_of_boards,level):
    if len(list_of_boards) == 0 or len(list_of_boards[0]) == 0 :        
        return
    
    for b in list_of_boards:
        b_notExist = get_element_position(b,9)
        if b_notExist is None:
            temp_b = Board(width=len(b[0]), height=len(b), 
                       initial_state=list(get_element_position(b,8)), 
                       goal_state=list(get_element_position(b,8)))
        else:
            temp_b = Board(width=len(b[0]), height=len(b), 
                           initial_state=list(get_element_position(b,8)), 
                           goal_state=list(get_element_position(b,9)))
        queue_.put([temp_b,level])
        
def concat_paths(main,solution):
    #Elminate tail
    #main.pop()
    #solution.pop()
    #Flip the solution path and concatenate it with the main path
    return np.concatenate((main, np.flip(solution,axis=0).tolist()), axis=0)

In [5]:
def goal_test(board):
    if np.array_equal(board.goal_state, board.initial_state):
        return True
    return False
    
def expand(board):
    global checked_positions
    expanded = []
    
    [i,j] = board.initial_state
    print(str(i)+" <- i /Expand/ j->"+str(j))
    act_pos = [i,j]
                # [V+,H+,V-,H-]
                # [up,right,down,left]
    movements = [i-1,j+1,i+1,j-1]
    names = ["ARRIBA","DERECHA","ABAJO","IZQUIERDA"]
    something = []
    
    for mov_index in range(0,4):
        move = movements[mov_index]
        temporal = np.array(board.board_state)
        if not(move > (board.width-1) or move < 0):
            #print(names[mov_index])
            #print("mov_index: "+str(mov_index)+" PASA: "+str(move)+" state[0,move]:"+str(t[0][move])+" state[move,0]:"+str(t[move][0]))
            #Vertical
            if(mov_index%2 == 0):
                #if where I want to move the position is empty
                if not temporal[move][j] in [0,9] or [move,j] in checked_positions or temporal[move][j] in [2]:
                    #print("Vertical mov. not valid Temp: "+str(temporal[move][j])+" [move,j]: "+str([move,j]))
                    continue
                something = swap_elements_by_position(temporal,act_pos,[move,j])
                something[act_pos[0]][act_pos[1]] = 1
                #print("Cae en SI "+str(mov_index)+" "+str(t))
            #Horizontal
            else:
                #if where I want to move the position is empty
                if not temporal[i][move] in [0,9] or [i,move] in checked_positions or temporal[i][move] in [2]:
                    #print("Horizontal mov. not valid Temp: "+str(temporal[move][j])+" [move,j]: "+str([move,j]))
                    continue
                something = swap_elements_by_position(temporal,act_pos,[i,move])
                something[act_pos[0]][act_pos[1]] = 1
                #print("Cae en NO "+str(mov_index)+" "+str(t))

            expanded.append(something)
            #print(expanded)
    return expanded
    

In [6]:
# BFS MAIN ALGORITHM
cost = 1 
def BFS(frontier):
    global cost
    act_board = frontier.get()
    print("\n"+str(cost)+". ==============\n"+str(act_board)+"\n=================")
    
    if goal_test(act_board):
        print("The BFS of the Labyrinth is: ")
        #print(act_board)
        return
    
    offspring = expand(act_board)
    print("===== offspring =====")
    print(str(np.array(list(offspring))))
    
    addtoQueue(frontier,offspring)
    
    #printQueue(frontier,30)
    cost+=1
    BFS(frontier)
    
# DFS MAIN ALGORITHM
def DFS(frontier):
    global cost
    act_board = frontier.get()
    print("\n"+str(cost)+". ==============\n"+str(act_board)+"\n=================")
    #print("act_element = "+str(act_element))
    
    if goal_test(act_board):
        print("The DFS of the Labyrinth is: ")
        print(act_board)
        return
    
    offspring = expand(act_board)
    offspring = shiftList(offspring)
    
    #print("offspring = "+str(offspring))

    addtoQueue(frontier,offspring)
    
    cost+=1
    DFS(frontier,goal)

# LDFS MAIN ALGORITHM
level = 0
def LDFS(frontier,limit):
    global cost, level
    act_state   = frontier.get()
    act_board   = act_state[0]
    level       = act_state[1]
    print("\n"+str(cost)+". ======== State | Level = "+str(level)+" ========")
    print(str(act_board))
    #print("act_element = "+str(act_element))
    
    if goal_test(act_board):
        print("The LDFS of the Labyrinth is: ")
        print(act_board)
        return
    if level >= limit:
        print("Level achieved: "+str(level)+" / "+str(limit))
        return
    level+=1
    offspring = expand(act_board)
    offspring = shiftList(offspring)
    
    #print("offspring = "+str(offspring))

    addtoQueueL(frontier,offspring,level)
    
    cost+=1
    
    LDFS(frontier,limit)

In [7]:
#BFS
def do_BFS(board):
    global cost
    print("=========== BFS ALGORITHM ===========")
    front = queue.Queue()
    front.put(board)
    BFS(front)
    print(str(cost)+" iterations were done.")
    cost = 0
    front.queue.clear()

def do_DFS(board):
    global cost
    print("=========== DFS ALGORITHM ===========")
    front = queue.LifoQueue()
    front.put(board)
    DFS(front)
    print(str(cost)+" iterations were done.")
    cost = 0
    front.queue.clear()

def do_LDFS(board,level_limit):
    global cost
    print("=========== LDFS ALGORITHM ===========")
    front = queue.LifoQueue()
    front.put([board,0])
    LDFS(front,level_limit)
    print(str(cost)+" iterations were done.")

    cost = 0
    front.queue.clear()

def do_ILDFS(board,level_limit,increment):
    global cost
    print("=========== ILDFS ALGORITHM ===========")
    front = queue.LifoQueue()
    front.put([board,0])

    iteration = 1

    while True:
        print("\n ======== ITERACIÓN "+str(iteration)+"========")
        LDFS(front,level_limit)
        level_limit += increment;
        print(str(cost)+" iterations were done.")

        iteration+=1
        cost = 0
        front.queue.clear()
        front.put([board,0])

In [8]:
# DATA INPUT

width = widgets.IntText(
    value=5,#int(input("Introduce the Width of the board: ")),
    description='Width:',
    disabled=True
)
height = widgets.IntText(
    value=10,#int(input("Introduce the Height of the board: ")),
    description='Height:',
    disabled=True
)
size = widgets.IntText(
    value=int(width.value*height.value),
    description='Size:',
    disabled=True
)

init_state = widgets.Box(children=
    [
        widgets.IntSlider(
            value=0,#int(m.floor(width.value/2)),
            min=0,
            max=width.value-1,
            step=1,
            description='X_width = ',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'),

        widgets.IntSlider(
            value=0,#int(m.floor(height.value/2)),
            min=0,
            max=height.value-1,
            step=1,
            description='Y_height = ',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
    )]
)

gol_state = widgets.Box(children=
    [
        widgets.IntSlider(
            value=3,#int(m.floor(width.value/2)),
            min=0,
            max=width.value-1,
            step=1,
            description='X_width = ',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'),

        widgets.IntSlider(
            value=3,#int(m.floor(height.value/2)),
            min=0,
            max=height.value-1,
            step=1,
            description='Y_height = ',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
    )]
)
                        
obstacle_ratio = widgets.Select(
    options=['5', '10', '15', '20','25','30'],
    value='10',
    description='Obstacle %:',
    disabled=False
)

search_algorithm = widgets.Select(
    options=['BFS', 'DFS', 'LDFS', 'ILDFS'],
    value='BFS',
    description='Algorithm:',
    disabled=False
)
level_limit = widgets.IntText(
    value = 2,
    description='LvL_limit:',
    disabled=False
)

increment = widgets.IntText(
    value = 2,
    description='LvL_limit:',
    disabled=False
)

wrapper_widget = widgets.Box([search_algorithm,obstacle_ratio,level_limit,increment])

board_options = widgets.Box(children=[widgets.Box([width, height, size])])

state_options = widgets.Tab(children=[init_state, gol_state])
state_options.set_title(0, 'Initial State')
state_options.set_title(1, 'Goal  State')

print("\n\n")
display(board_options)
display(state_options)
display(wrapper_widget)

#display(search_algorithm)
#display(obstacle_ratio)

Box(children=(Box(children=(IntText(value=5, description='Width:', disabled=True), IntText(value=10, descripti…

Box(children=(Select(description='Algorithm:', options=('BFS', 'DFS', 'LDFS', 'ILDFS'), value='BFS'), Select(d…

In [9]:
#Parsing of the inputs
p_width = int(width.value)
p_height = int(height.value)
p_init_state = [init_state.children[1].value,init_state.children[0].value]
p_goal_state = [gol_state.children[1].value,gol_state.children[0].value]
p_obstacle_ratio = int(obstacle_ratio.value)
p_search_algorithm = search_algorithm.value
p_lvl_limit = int(level_limit.value)
p_increment = int(increment.value)
#print(type(p_obstacle_ratio))

print(p_width)
print(p_height)

print(p_init_state)
print(p_goal_state)

print(p_obstacle_ratio)
print(p_search_algorithm)
print(p_obstacle_ratio)
print(p_lvl_limit)

5
10
[0, 0]
[3, 3]
10
BFS
10
2


In [10]:
#BOARD CREATION AND OBSTACLES 

board = Board(width = p_width,
              height = p_height,
              initial_state = p_init_state,
              goal_state = p_goal_state,
             )
board.set_random_obstacles(p_obstacle_ratio)

print(board)
#print(board)

#board = ''

algo_options = ['BFS', 'DFS', 'LDFS', 'ILDFS']
algorithm = algo_options.index(p_search_algorithm)

if algorithm == 0:
    do_BFS(board)
if algorithm == 1:
    do_DFS(board)
if algorithm == 2:
    do_LDFS(board,p_lvl_limit)
if algorithm == 3:
    do_ILDFS(board,p_lvl_limit,p_increment)

#algo_method  = [do_BFS(board),do_DFS(),do_LDFS(),do_ILDFS()]

checked_positions = []
obstacles_positions = []
board = ''
#print(board)
#print(p.isin([1]))
#print(np.array(temp_board[temp_board.isin([2])].count()).sum()) #Obtener elementos que tengan y sumarlos
#print([[0]*width]*height)
#p = pd.DataFrame([[0]*width]*height)
#p.loc[0:2,0:2] = 1

[[0, 0]]
n_obstacles = 5


,0,1,2,3,4
0,8,0,2,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,9,0
4,0,0,0,0,2
5,0,0,0,0,0
6,0,0,0,0,2
7,0,0,0,0,0
8,2,0,0,0,0
9,0,0,0,2,0


   0  1  2  3  4
0  8  0  2  0  0
1  0  0  0  0  0
2  0  0  0  0  0
3  0  0  0  9  0
4  0  0  0  0  2
5  0  0  0  0  0
6  0  0  0  0  2
7  0  0  0  0  0
8  2  0  0  0  0
9  0  0  0  2  0
=========== BFS ALGORITHM ===========


,0,1,2,3,4
0,8,0,2,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,9,0
4,0,0,0,0,2
5,0,0,0,0,0
6,0,0,0,0,2
7,0,0,0,0,0
8,2,0,0,0,0
9,0,0,0,2,0



1. ==============
   0  1  2  3  4
0  8  0  2  0  0
1  0  0  0  0  0
2  0  0  0  0  0
3  0  0  0  9  0
4  0  0  0  0  2
5  0  0  0  0  0
6  0  0  0  0  2
7  0  0  0  0  0
8  2  0  0  0  0
9  0  0  0  2  0
0 <- i /Expand/ j->0
===== offspring =====
[[[1 8 2 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 9 0]
  [0 0 0 0 2]
  [0 0 0 0 0]
  [0 0 0 0 2]
  [0 0 0 0 0]
  [2 0 0 0 0]
  [0 0 0 2 0]]

 [[1 0 2 0 0]
  [8 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 9 0]
  [0 0 0 0 2]
  [0 0 0 0 0]
  [0 0 0 0 2]
  [0 0 0 0 0]
  [2 0 0 0 0]
  [0 0 0 2 0]]]
[[0, 0], [0, 0], [0, 0], [0, 1]]
[[0, 0], [0, 0], [0, 0], [0, 1], [1, 0]]


...


19. ==============
   0  1  2  3  4
0  1  1  2  1  1
1  1  1  1  1  1
2  1  1  1  1  8
3  1  1  1  9  0
4  1  1  1  0  2
5  0  0  0  0  0
6  0  0  0  0  2
7  0  0  0  0  0
8  2  0  0  0  0
9  0  0  0  2  0
2 <- i /Expand/ j->4
===== offspring =====
[[[1 1 2 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 9 8]
  [1 1 1 0 2]
  [0 0 0 0 0]
  [0 0 0 0 2]
  [0 0 0 0 0]
  [2 0 0 0 0]
  [0 0 0 2 0]]]
[[0, 0], [0, 0], [0, 0], [0, 1], [1, 0], [0, 1], [1, 1], [1, 0], [2, 0], [1, 1], [1, 2], [2, 1], [2, 0], [3, 0], [1, 2], [1, 3], [2, 2], [2, 1], [3, 1], [3, 0], [4, 0], [1, 3], [0, 3], [1, 4], [2, 3], [2, 2], [3, 2], [3, 1], [4, 1], [4, 0], [0, 3], [0, 4], [1, 4], [2, 4], [2, 3], [3, 3], [3, 2], [4, 2], [4, 1], [0, 4], [2, 4], [3, 4]]


,0,1,2,3,4
0,1,1,2,1,1
1,1,1,1,1,1
2,1,1,1,1,1
3,1,1,1,9,1
4,1,1,1,0,2
5,0,0,0,0,0
6,0,0,0,0,2
7,0,0,0,0,0
8,2,0,0,0,0
9,0,0,0,2,0



20. ==============
   0  1  2  3  4
0  1  1  2  1  1
1  1  1  1  1  1
2  1  1  1  1  1
3  1  1  1  9  1
4  1  1  1  0  2
5  0  0  0  0  0
6  0  0  0  0  2
7  0  0  0  0  0
8  2  0  0  0  0
9  0  0  0  2  0
The BFS of the Labyrinth is: 


,0,1,2,3,4
0,1,1,2,1,1
1,1,1,1,1,1
2,1,1,1,1,1
3,1,1,1,9,1
4,1,1,1,0,2
5,0,0,0,0,0
6,0,0,0,0,2
7,0,0,0,0,0
8,2,0,0,0,0
9,0,0,0,2,0


   0  1  2  3  4
0  1  1  2  1  1
1  1  1  1  1  1
2  1  1  1  1  1
3  1  1  1  9  1
4  1  1  1  0  2
5  0  0  0  0  0
6  0  0  0  0  2
7  0  0  0  0  0
8  2  0  0  0  0
9  0  0  0  2  0
20 iterations were done.
